In [1]:
!pip install pyarrow

In [2]:
import pandas as pd

df_jan = pd.read_parquet("yellow_tripdata_2023-01.parquet")
df_feb = pd.read_parquet("yellow_tripdata_2023-02.parquet")
df_feb.shape[1]

19

In [3]:
len(df_jan.columns)

19

In [4]:
df_jan.columns == df_feb.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False])

In [5]:
df_jan.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [6]:
df_feb.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee'],
      dtype='object')

In [7]:
df_feb.rename(columns={"Airport_fee": "airport_fee"}, inplace=True)

In [8]:
df_jan.columns == df_feb.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [9]:
(
    df_jan["tpep_pickup_datetime"].isna().sum()
    + df_feb["tpep_pickup_datetime"].isna().sum()
)

0

In [10]:
(
    df_jan["tpep_dropoff_datetime"].isna().sum()
    + df_feb["tpep_dropoff_datetime"].isna().sum()
)

0

In [11]:
duration_all = df_jan["tpep_dropoff_datetime"] - df_jan["tpep_pickup_datetime"]
duration_all.std()

Timedelta('0 days 00:42:35.661074')

In [12]:
(
    df_jan[["DOLocationID", "PULocationID"]].isna().sum()
    + df_feb[["DOLocationID", "PULocationID"]].isna().sum()
)

DOLocationID    0
PULocationID    0
dtype: int64

In [13]:
filtered_df = df_jan[
    (
        (df_jan["tpep_dropoff_datetime"] - df_jan["tpep_pickup_datetime"])
        >= pd.Timedelta(minutes=1)
    )
    & (
        (df_jan["tpep_dropoff_datetime"] - df_jan["tpep_pickup_datetime"])
        <= pd.Timedelta(minutes=60)
    )
]

filtered_df_val = df_feb[
    (
        (df_feb["tpep_dropoff_datetime"] - df_feb["tpep_pickup_datetime"])
        >= pd.Timedelta(minutes=1)
    )
    & (
        (df_feb["tpep_dropoff_datetime"] - df_feb["tpep_pickup_datetime"])
        <= pd.Timedelta(minutes=60)
    )
]

In [14]:
duration = filtered_df["tpep_dropoff_datetime"] - filtered_df["tpep_pickup_datetime"]
duration_val = (
    filtered_df_val["tpep_dropoff_datetime"] - filtered_df_val["tpep_pickup_datetime"]
)

In [15]:
duration

0         0 days 00:08:26
1         0 days 00:06:19
2         0 days 00:12:45
3         0 days 00:09:37
4         0 days 00:10:50
                ...      
3066761   0 days 00:13:59
3066762   0 days 00:19:27
3066763   0 days 00:24:31
3066764   0 days 00:13:00
3066765   0 days 00:14:24
Length: 3009173, dtype: timedelta64[us]

In [16]:
duration.std()

Timedelta('0 days 00:09:56.363137')

In [17]:
len(duration) / len(duration_all)

0.9812202822125979

In [18]:
filtered_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [19]:
from sklearn.feature_extraction import DictVectorizer

# convert location IDs to strings
filtered_df = filtered_df.assign(PULocationID=filtered_df["PULocationID"].astype(str))
filtered_df = filtered_df.assign(DOLocationID=filtered_df["DOLocationID"].astype(str))
filtered_df_val = filtered_df_val.assign(
    PULocationID=filtered_df_val["PULocationID"].astype(str)
)
filtered_df_val = filtered_df_val.assign(
    DOLocationID=filtered_df_val["DOLocationID"].astype(str)
)

In [20]:
# combining to ensure all categories are known
X_train = filtered_df[["PULocationID", "DOLocationID"]]
X_val = filtered_df_val[["PULocationID", "DOLocationID"]]
combined_dicts = pd.concat([X_train, X_val]).to_dict(orient="records")

vectorizer = DictVectorizer(sparse=True)
vectorizer.fit(combined_dicts)  # Learn all features from combined data

# Transform datasets separately
X_train_vectorized = vectorizer.transform(X_train.to_dict(orient="records"))
X_val_vectorized = vectorizer.transform(X_val.to_dict(orient="records"))

In [21]:
print("Feature names:", vectorizer.get_feature_names_out([X_train_vectorized]))

Feature names: ['DOLocationID=1' 'DOLocationID=10' 'DOLocationID=100' 'DOLocationID=101'
 'DOLocationID=102' 'DOLocationID=105' 'DOLocationID=106'
 'DOLocationID=107' 'DOLocationID=108' 'DOLocationID=109'
 'DOLocationID=11' 'DOLocationID=111' 'DOLocationID=112'
 'DOLocationID=113' 'DOLocationID=114' 'DOLocationID=115'
 'DOLocationID=116' 'DOLocationID=117' 'DOLocationID=118'
 'DOLocationID=119' 'DOLocationID=12' 'DOLocationID=120'
 'DOLocationID=121' 'DOLocationID=122' 'DOLocationID=123'
 'DOLocationID=124' 'DOLocationID=125' 'DOLocationID=126'
 'DOLocationID=127' 'DOLocationID=128' 'DOLocationID=129'
 'DOLocationID=13' 'DOLocationID=130' 'DOLocationID=131'
 'DOLocationID=132' 'DOLocationID=133' 'DOLocationID=134'
 'DOLocationID=135' 'DOLocationID=136' 'DOLocationID=137'
 'DOLocationID=138' 'DOLocationID=139' 'DOLocationID=14'
 'DOLocationID=140' 'DOLocationID=141' 'DOLocationID=142'
 'DOLocationID=143' 'DOLocationID=144' 'DOLocationID=145'
 'DOLocationID=146' 'DOLocationID=147' 'DOLoc

In [22]:
X_train_vectorized.shape[1]

519

In [23]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [24]:
target = duration.dt.total_seconds() / 60.0
target

0           8.433333
1           6.316667
2          12.750000
3           9.616667
4          10.833333
             ...    
3066761    13.983333
3066762    19.450000
3066763    24.516667
3066764    13.000000
3066765    14.400000
Length: 3009173, dtype: float64

In [25]:
model = LinearRegression()
model.fit(X_train_vectorized, target)

LinearRegression()

In [26]:
predictions = model.predict(X_train_vectorized)
predictions

array([11.52744385, 10.89786759, 11.32577815, ..., 11.73771364,
       12.70533125, 11.54221663])

In [27]:
# Calculate RMSE on train
rmse = np.sqrt(mean_squared_error(target, predictions))
print("RMSE on the training data:", rmse)

RMSE on the training data: 7.6492619310517975


In [28]:
predictions_val = model.predict(X_val_vectorized)
target_val = duration_val.dt.total_seconds() / 60.0

In [29]:
predictions_val

array([11.36782977, 49.69480593, 15.18872621, ..., 13.53316151,
       11.09582578, 13.15710349])

In [30]:
# Calculate RMSE on validation
rmse_val = np.sqrt(mean_squared_error(target_val, predictions_val))
print("RMSE on the training data:", rmse_val)

RMSE on the training data: 7.811821290624478
